In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=e6a7c443f9fafd21b5eca28715e79f0fdb8d9ada0abac20470fc5e0c1bd3e14d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


# Pandas

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark import *

In [3]:
df_pd = pd.read_csv('https://raw.githubusercontent.com/bahdanparakevich/pyspark_tests/14688593773d42cf73525c35f6ce80834588bd65/raw_data/DataEngineer.csv')

In [204]:
df_pd.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,Data Engineer,$80K-$150K (Glassdoor est.),Company Description\nSagence is a management a...,4.5,Sagence\n4.5,"New York, NY","Chicago, IL",1 to 50 employees,2009,Company - Private,Consulting,Business Services,$10 to $25 million (USD),"WCI Consulting, PwC",-1
1,Senior Data Engineer (Healthcare Domain experi...,$80K-$150K (Glassdoor est.),"Key Responsibilities\n\n- Architect, build, an...",3.4,Enterprise Integration\n3.4,"New York, NY","Jacksonville, FL",51 to 200 employees,1998,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),-1,-1
2,Data Engineers,$80K-$150K (Glassdoor est.),Overview\n\nJob description\n\nPosition Overvi...,5.0,Maestro Technologies\n5.0,"New York, NY","Trenton, NJ",51 to 200 employees,2003,Company - Private,IT Services,Information Technology,$5 to $10 million (USD),-1,-1
3,Client Trade Support Engineer,$80K-$150K (Glassdoor est.),About the Position\n\n\nThis position will sup...,4.8,Jane Street\n4.8,"New York, NY","New York, NY",501 to 1000 employees,2000,Company - Private,Investment Banking & Asset Management,Finance,Unknown / Non-Applicable,-1,-1
4,Data Engineer,$80K-$150K (Glassdoor est.),Data Engineer\n\nJob Details\nLevel\nExperienc...,3.7,GNY Insurance Companies\n3.7,"New York, NY","New York, NY",201 to 500 employees,1914,Company - Private,Insurance Carriers,Insurance,$100 to $500 million (USD),"Travelers, Chubb, Crum & Forster",True


Number of rows


In [164]:
df_pd.shape[0]

2528

The oldest Company in the Dataframe


In [167]:
min(df_pd.Founded.loc[df_pd.Founded>0])

1625

The newest company in the Dataframe


In [168]:
max(df_pd.Founded)

2020

Number of vacancies in NY

In [170]:
df_pd.loc[df_pd.Location=='New York, NY'].shape[0]

187

Number of sectors

In [171]:
df_pd.Sector.nunique()

25

Number of companies with less then 200 employees

In [173]:
df_pd.loc[(df_pd.Size=='1 to 50 employees') | (df_pd.Size=='51 to 200 employees')]['Company Name'].nunique()

530

Minimum of salary

In [190]:
min([int(str(i)[:-19].split('-')[0][1:-1]) for i in df_pd['Salary Estimate'].unique()])

27

Maximum of salary

In [211]:
max([int(str(i).split('-')[1][:-17][1:-1]) for i in df_pd['Salary Estimate'].unique()])

281

# PySpark

In [40]:
from pyspark.sql.functions import count_distinct

In [158]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read.option("delimiter", ",").option("quote", "\"").option("multiLine", True).option("escape", "\"").csv("/content/DataEngineer (1).csv", header=True, inferSchema=True)

df.show()

+--------------------+--------------------+--------------------+------+--------------------+---------------+--------------------+--------------------+-------+-----------------+--------------------+--------------------+--------------------+--------------------+----------+
|           Job Title|     Salary Estimate|     Job Description|Rating|        Company Name|       Location|        Headquarters|                Size|Founded|Type of ownership|            Industry|              Sector|             Revenue|         Competitors|Easy Apply|
+--------------------+--------------------+--------------------+------+--------------------+---------------+--------------------+--------------------+-------+-----------------+--------------------+--------------------+--------------------+--------------------+----------+
|       Data Engineer|$80K-$150K (Glass...|Company Descripti...|   4.5|        Sagence\n4.5|   New York, NY|         Chicago, IL|   1 to 50 employees|   2009|Company - Private|        

In [103]:
df.schema

StructType([StructField('Job Title', StringType(), True), StructField('Salary Estimate', StringType(), True), StructField('Job Description', StringType(), True), StructField('Rating', DoubleType(), True), StructField('Company Name', StringType(), True), StructField('Location', StringType(), True), StructField('Headquarters', StringType(), True), StructField('Size', StringType(), True), StructField('Founded', IntegerType(), True), StructField('Type of ownership', StringType(), True), StructField('Industry', StringType(), True), StructField('Sector', StringType(), True), StructField('Revenue', StringType(), True), StructField('Competitors', StringType(), True), StructField('Easy Apply', StringType(), True)])

Number of rows


In [163]:
df.count()

2528

The oldest Company in the Dataframe


In [104]:
df.filter(df.Founded>0).agg(F.min('Founded')).collect()[0][0]

1625

The newest company in the Dataframe


In [105]:
df.filter(df.Founded>0).agg(F.max('Founded')).collect()[0][0]

2020

Number of vacancies in NY

In [107]:
df.filter(df.Location=='New York, NY').count()

187

Number of sectors

In [119]:
df.select('Sector').distinct().count()

25

In [117]:
df.select(count_distinct(df.Sector)).show()

+----------------------+
|count(DISTINCT Sector)|
+----------------------+
|                    25|
+----------------------+



Number of companies with less then 200 employees

In [123]:
df.filter((df.Size=='1 to 50 employees') | (df.Size=='51 to 200 employees')).select('Company Name').distinct().count()

530

Minimum of salary

In [153]:
min([int(str(i)[21:-19].split('-')[0][1:-1]) for i in df.select('Salary Estimate').distinct().collect()])

27

Maximum of salary

In [196]:
max([int(str(i)[21:-19].split('-')[1][1:-1]) for i in df.select('Salary Estimate').distinct().collect()])

281